In [18]:
!git clone https://github.com/Oluchi-Otuadinma/Sexism-Classification.git
%cd /content/Sexism-Classification

fatal: destination path 'Sexism-Classification' already exists and is not an empty directory.
/content/Sexism-Classification


In [22]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 959 bytes | 959.00 KiB/s, done.
From https://github.com/Oluchi-Otuadinma/Sexism-Classification
   a640332..e78de4c  main       -> origin/main
Updating a640332..e78de4c
Fast-forward
 requirements.txt | 1 +
 1 file changed, 1 insertion(+)


In [19]:
!ls

data	 notebooks  requirements.txt	   src
LICENSE  README.md  Sexism-Classification


In [23]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 3.2 MB/s eta 0:00:00


In [9]:
import sys
sys.path.append('/content/Sexism-Classification/src')

In [10]:
from data.load_data import load_raw_csvs

PROJECT_PATH = "/content/Sexism-Classification"
RAW_PATH = f"{PROJECT_PATH}/data/raw"

dfs = load_raw_csvs(RAW_PATH)

In [11]:
train = dfs['train']
test = dfs['test']
dev = dfs['dev']

In [12]:
list(train.columns)

['rewire_id',
 'text',
 'label_sexist',
 'label_category',
 'label_vector',
 'split']

In [25]:
import pandera.pandas as pa

schema = pa.DataFrameSchema({
     'rewire_id': pa.Column(str),
     'text': pa.Column(str),
     'label_sexist': pa.Column(str),
     'label_category': pa.Column(str),
     'label_vector': pa.Column(str),
     'split': pa.Column(str),
 })

In [28]:
schema.validate(train)
schema.validate(test)
schema.validate(dev)

,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev
1,sexism2022_english-966,I bet she wished she had a gun,not sexist,none,none,dev
2,sexism2022_english-3553,I agree with that but at the same time I know ...,sexist,2. derogation,2.1 descriptive attacks,dev
3,sexism2022_english-13328,democrats from Minnesota (leftist muzzie shith...,sexist,2. derogation,2.2 aggressive and emotive attacks,dev
4,sexism2022_english-81,Texas jury sentences Jordanian immigrant to de...,not sexist,none,none,dev
...,...,...,...,...,...,...
1995,sexism2022_english-6672,The future is female guys,not sexist,none,none,dev
1996,sexism2022_english-12605,Dude do not ever text a girl something like th...,not sexist,none,none,dev
1997,sexism2022_english-10008,I cum really hard from anal & clitoral stimula...,not sexist,none,none,dev
1998,sexism2022_english-17035,Oooh that hurt kind like that Jewish girl who ...,not sexist,none,none,dev


In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


# Initialise NLP tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to clean text and extract domains
def clean_text(text, lowercase=True, replace_urls=True, extract_domain=False, remove_stopwords=True, lemmatize=True):
    if lowercase:
        text = text.lower()

    # Extract domain names from URLs
    url_pattern = r'https?://\S+|www\.\S+'
    domains = []  # Store extracted domains
    matches = re.findall(url_pattern, text)

    for match in matches:
        extracted = tldextract.extract(match)
        domain = f"{extracted.domain}.{extracted.suffix}"  # e.g., "cnn.com"
        domains.append(domain)  # Save domain for analysis
        text = text.replace(match, "")  # Remove the URL from text

    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    if remove_stopwords:
        tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    if lemmatize:
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens), domains  # Convert tokens back to string